# Feature-Extraction
## Theoretische Grundlagen, Methodik und Umsetzung
### KI in der Medizin: Brustkrebs Diagnose

Wir haben den öffentlich verfügbaren Datensatz Breast Cancer Wisconsin verwendet und vom UCI Machine Learning Repository heruntergeladen. 

Repository: https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29


![](./data.png)

Unser Ziel ist es, herauszufinden, welche Merkmale bei der Vorhersage von bösartigem oder gutartigem Krebs am hilfreichsten sind, und zu klassifizieren, ob der Brustkrebs gutartig oder bösartig ist.

Wir haben den öffentlich verfügbaren Datensatz Breast Cancer Wisconsin verwendet und vom UCI Machine Learning Repository heruntergeladen.

Die typische Leistungsanalyse wird durchgeführt

![](conf_matrix.png)

In [ ]:
# Bibliotheken importieren
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from cf_matrix import make_confusion_matrix
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, StratifiedKFold, GridSearchCV, cross_validate
import matplotlib.pylab as pl
import pandas as pd
import numpy as np
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [ ]:
# Download des Krebs-Datensatzes 
import seaborn as sns
from sklearn import preprocessing
(X, y) = load_breast_cancer(return_X_y=True, as_frame=True)

### Baseline: Entscheidungsbäumen Klassifikator

In [ ]:
# Baseline in der Performance mit Entscheidungsbäumen
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from cf_matrix import make_confusion_matrix
# Daten Skalierung
t = MinMaxScaler()
t.fit(X)
X = t.transform(X)
# Einfacher binärer Klassifikator
model = XGBClassifier()
#  definieren Sie das Verfahren der Kreuzvalidierung
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
# Modell auswerten
scores = cross_validate(estimator=model, X=X, y=y, cv=cv, n_jobs=-1, 
                        scoring=['accuracy', 'roc_auc', 'precision', 'recall', 'f1'])
print('Accuracy: ', scores['test_accuracy'].mean())
print('Precision: ', scores['test_precision'].mean())
print('Recall: ', scores['test_recall'].mean())
print('F1: ', scores['test_f1'].mean(), '\n')

# Manuelle Feature-Extraction

## Korrelation

Oft haben wir Features, die hoch korreliert sind und somit redundante Informationen liefern. Durch die Eliminierung hoch korrelierter Features können wir eine Vorhersageverzerrung für die in diesen Features enthaltenen Informationen vermeiden. 
Wir entfernen alle Merkmale mit einer Korrelation von mehr als 0,7.

![](pearson_korr.png)

In [ ]:
# Download des Krebs-Datensatzes 
import seaborn as sns
from sklearn import preprocessing
(X, y) = load_breast_cancer(return_X_y=True, as_frame=True)

In [ ]:
# Korrelationanalyse
X = X.iloc[:,1:-1]
label_encoder = preprocessing.LabelEncoder()
X.iloc[:,0] = label_encoder.fit_transform(X.iloc[:,0]).astype('float64')
corr = X.corr()
sns.heatmap(corr)
columns = np.full((corr.shape[0],), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= 0.7:
            if columns[j]:
                columns[j] = False
selected_columns = X.columns[columns]
X = X[selected_columns]

In [ ]:
# Überblick über die Daten
X

In [ ]:
# Baseline in der Performance mit XGBoost Modell
# Einfacher binärer Klassifikator
model = XGBClassifier()
#  definieren Sie das Verfahren der Kreuzvalidierung
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
# Modell auswerten
scores = cross_validate(estimator=model, X=X, y=y, cv=cv, n_jobs=-1, 
                        scoring=['accuracy', 'roc_auc', 'precision', 'recall', 'f1'])
print('Accuracy: ', scores['test_accuracy'].mean())
print('Precision: ', scores['test_precision'].mean())
print('Recall: ', scores['test_recall'].mean())
print('F1: ', scores['test_f1'].mean(), '\n')

# Automatic Feature Extraction

## Autoencoder 

In [ ]:
# Datensatz in Trainings- und Testdatensatz aufteilen
(X, y) = load_breast_cancer(return_X_y=True, as_frame=True)
X_train, X_test, t_train, y_test = train_test_split(X, y)

In [ ]:
# Erstellen und Trainieren eines Autoencoders für die Klassifizierung mit Kompression in der Bottleneck-Schicht.
# Anzahl der Eingangsspalten
n_inputs = X.shape[1]
# Skalierung der Daten
t = MinMaxScaler()
t.fit(X_train)
X_train = t.transform(X_train)
X_test = t.transform(X_test)
# Encoder definieren
visible = Input(shape=(n_inputs,))
# Encoder Ebene 1
e = Dense(n_inputs*2)(visible)
e = BatchNormalization()(e)
e = LeakyReLU()(e)
# Encoder Ebene 2
e = Dense(n_inputs)(e)
e = BatchNormalization()(e)
e = LeakyReLU()(e)
# Bottleneck-Schicht
n_bottleneck = round(float(n_inputs) / 2.0)
bottleneck = Dense(n_bottleneck)(e)
# Decoder definieren, Ebene 1
d = Dense(n_inputs)(bottleneck)
d = BatchNormalization()(d)
d = LeakyReLU()(d)
# Decoder Ebene 1
d = Dense(n_inputs*2)(d)
d = BatchNormalization()(d)
d = LeakyReLU()(d)
# Ausgabe
output = Dense(n_inputs, activation='linear')(d)

### Aufbau des Autoencoders
![](autoencoder.png)

In [ ]:
# Autoencoder definieren
model = Model(inputs=visible, outputs=output)
# Kompilieren des Autoencoders
model.compile(optimizer='adam', loss='mse')
# Fit das Autoencoder-Modell, um die Eingabe zu rekonstruieren
history = model.fit(X_train, X_train, epochs=200, batch_size=10, verbose=2, validation_data=(X_test,X_test))
# die Entwicklung der Verlustfunktion darstellen
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'], label='Testing')
plt.legend()
plt.show()
# das Encodermodell vom Autoencoder trennen (ohne den Decoder)
encoder = Model(inputs=visible, outputs=bottleneck)
# Speichern des Encoders
encoder.save('encoder.h5')

### Aufbau des Encoders
![](encoder.png)

### Encoder + Entscheidungsbäumen Klassifikator

In [ ]:
# Entscheidungsbäumen auf kodierter Eingabe auswerten (Encoder + Entscheidungsbäumen Klassifikator)
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model
# Laden des Modells aus Datei
encoder = load_model('encoder.h5')
model = XGBClassifier()
# die Trainingsdaten kodieren
X_train_encode = encoder.predict(X)
#  definieren Sie das Verfahren der Kreuzvalidierung
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
# Modell auswerten
scores = cross_validate(estimator=model, X=X_train_encode, y=y, cv=cv, n_jobs=-1, 
                        scoring=['accuracy', 'roc_auc', 'precision', 'recall', 'f1'])
print('Accuracy: ', scores['test_accuracy'].mean())
print('Precision: ', scores['test_precision'].mean())
print('Recall: ', scores['test_recall'].mean())
print('F1: ', scores['test_f1'].mean(), '\n')

### PCA + Entscheidungsbäumen Klassifikator

In [ ]:
# Klassifizierung auf reduzierter Dimension (PCA-Merkmalsextrahierung + Entscheidungsbäumenklassifikator)
from sklearn.decomposition import PCA
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
# Liste der Transformationen an den Daten
transforms = list()
transforms.append(('pca', PCA(n_components=9)))
fu = FeatureUnion(transforms)
# das Modell definieren
model = XGBClassifier()
# die Pipeline definieren
steps = list()
steps.append(('fu', fu))
steps.append(('m', model))
pipeline = Pipeline(steps=steps)
#  definieren Sie das Verfahren der Kreuzvalidierung
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
# Modell auswerten
scores = cross_validate(estimator=pipeline, X=X, y=y, cv=cv, n_jobs=-1, 
                        scoring=['accuracy', 'roc_auc', 'precision', 'recall', 'f1'])
print('Accuracy: ', scores['test_accuracy'].mean())
print('Precision: ', scores['test_precision'].mean())
print('Recall: ', scores['test_recall'].mean())
print('F1: ', scores['test_f1'].mean(), '\n')

Über diesen Github-Link können Sie auf den gesamten Code und die Daten des Projekts zugreifen.

![](download.png)